![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBERT.ipynb)

# Import ONNX CamemBERT models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for CamemBERT from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use CamemBERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [camembert-base](https://huggingface.co/camembert-base) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the CamemBERT model, we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import CamembertTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "camembert-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

('onnx_models/camembert-base/tokenizer_config.json',
 'onnx_models/camembert-base/special_tokens_map.json',
 'onnx_models/camembert-base/sentencepiece.bpe.model',
 'onnx_models/camembert-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 430892
-rw-r--r-- 1 root root        22 Jun 10 20:15 added_tokens.json
-rw-r--r-- 1 root root       636 Jun 10 20:15 config.json
-rw-r--r-- 1 root root 440405686 Jun 10 20:15 model.onnx
-rw-r--r-- 1 root root    810912 Jun 10 20:15 sentencepiece.bpe.model
-rw-r--r-- 1 root root       374 Jun 10 20:15 special_tokens_map.json
-rw-r--r-- 1 root root      1814 Jun 10 20:15 tokenizer_config.json


- We need to move the `sentencepiece.bpe.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Camembert or other SentencePiece-based tokenizers.

In [4]:
!mkdir -p {EXPORT_PATH}/assets & mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 792
-rw-r--r-- 1 root root 810912 Jun 10 20:15 sentencepiece.bpe.model


Voila! We have our `sentencepiece.bpe.model` inside assets directory

## Import and Save CamemBERT in Spark NLP

- Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 47.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `CamemBertEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `CamemBertEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import CamemBertEmbeddings

camembert = CamemBertEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("camembert")\
    .setCaseSensitive(True)\
    .setDimension(768)\
    .setStorageRef('camembert_base')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
camembert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX CamemBERT model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 430952
-rw-r--r-- 1 root root 440473025 Jun 10 20:19 camembert_onnx
-rw-r--r-- 1 root root    810912 Jun 10 20:19 camembert_spp
drwxr-xr-x 2 root root      4096 Jun 10 20:19 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBERT model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

camembert_loaded = CamemBertEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("camembert")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    camembert_loaded
])

data = spark.createDataFrame([
    ["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."]
]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)
result.selectExpr("explode(camembert.embeddings) as embeddings").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                          embeddings|
+----------------------------------------------------------------------------------------------------+
|[-0.048798718, 0.13934422, 0.17626338, 0.15536329, -0.016455, -0.012065249, -0.13586293, 0.042687...|
|[0.007033877, 0.15960854, 0.11289671, 0.12562779, -0.015402641, -0.21577366, -0.07879828, 0.09592...|
|[-0.021704827, -0.1526426, -0.068746425, 0.19136184, -0.10606898, -0.08022556, -0.171636, -0.0090...|
|[0.045735035, 0.09138474, -0.19595747, 0.024617488, 0.0609179, 0.06903413, -0.104803, -0.05503395...|
|[0.092956275, 0.07005064, -0.1215685, 0.1707416, 0.06562891, 0.12712705, -0.05587231, -0.05288334...|
|[0.06815244, 0.22697632, -0.12149057, 0.01782827, 0.05155113, 0.021605168, -0.13541833, -0.232393...|
|[0.08471602, -0.33922884, -0.07471369, 0.23726627, -0.03673842, -0.17772

That's it! You can now go wild and use hundreds of CamemBERT models from HuggingFace 🤗 in Spark NLP 🚀
